In [ ]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
import keras
from keras import layers

from keras import regularizers

from keras.layers import  Dropout, Flatten, MaxPooling2D, Bidirectional, CuDNNLSTM
from keras import regularizers
from keras.models import Sequential
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv(r'D:\Jassi\freesound-audio-tagging/train.csv')
label = train['label'].unique()

In [ ]:
names = train.groupby('label')['fname']
names 

In [ ]:
#from one directory to labeled directory audio files
import pathlib 
import os 
import shutil 

filename =   list(label)

for i in filename:
    pathlib.Path(f"/content/drive/My Drive/Data/freesound-audio-tagging/audio_train_labeled/{i}/").mkdir(parents=True, exist_ok=True)
    os.listdir(f"/content/drive/My Drive/Data/freesound-audio-tagging/audio_train/")
    for y in names.get_group(i):
        shutil.move( f"/content/drive/My Drive/Data/freesound-audio-tagging/audio_train/{y}" , f"/content/drive/My Drive/Data/freesound-audio-tagging/audio_train_labeled/{i}/")



In [ ]:
#diff directory img data
#create image files
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
filename =   list(label)
for i in filename:
    pathlib.Path(f'D:\Jassi/freesound-audio-tagging/freesound_img_data/{i}').mkdir(parents=True, exist_ok=True)
    for files in os.listdir(f'D:\Jassi/freesound-audio-tagging/audio_train_labeled/{i}'):
        songname = f'D:\Jassi/freesound-audio-tagging/audio_train_labeled/{i}/{files}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'D:\Jassi/freesound-audio-tagging/freesound_img_data/{i}/{files[:-3].replace(".", "")}.png')
        plt.clf()

In [ ]:
#same directory
#create image files
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
filename =   list(label)

pathlib.Path(f'D:\Jassi/freesound-audio-tagging/freesound_img_data_2/').mkdir(parents=True, exist_ok=True)
for files in os.listdir(f'D:\Jassi/freesound-audio-tagging/audio_train/'):
    songname = f'D:\Jassi/freesound-audio-tagging/audio_train/{files}'
    y, sr = librosa.load(songname, mono=True, duration=5)
    plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
    plt.axis('off');
    plt.savefig(f'D:\Jassi/freesound-audio-tagging/freesound_img_data_2/{files[:-3].replace(".", "")}.png')
    plt.clf()

In [ ]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate nframes'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [ ]:
#for data in diff directory
#create csv file and add features to it
file = open(r'D:\Jassi\freesound-audio-tagging\dataset.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
filename =   list(label)
for i in filename:
    for files in os.listdir(f'D:/Jassi/freesound-audio-tagging/audio_train_labeled/{i}'):
        songname = f'D:/Jassi/freesound-audio-tagging/audio_train_labeled/{i}/{files}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        rmse = librosa.feature.rms(y=y)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{files} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {i}'
        file = open(f"D:/Jassi/freesound-audio-tagging/dataset.csv", 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [ ]:
#data at same directory
#create csv file and add more features to it
file = open(r'D:\Jassi\freesound-audio-tagging\dataset2.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

for files in os.listdir(f'D:/Jassi/freesound-audio-tagging/audio_train/'):
    songname = f'D:/Jassi/freesound-audio-tagging/audio_train/{files}'
    y, sr = librosa.load(songname, mono=True, duration=30)
    rmse = librosa.feature.rms(y=y)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
       
    import wave
    nframes = librosa.util.frame(y)
        
    to_append = f'{files} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)} {np.mean(nframes)}'    
    for e in mfcc:
        to_append += f' {np.mean(e)}'
    file = open(f"D:/Jassi/freesound-audio-tagging/dataset2.csv", 'a', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(to_append.split())


In [ ]:
df = pd.DataFrame(train.iloc[:, -2])
df

In [ ]:
df1 = pd.read_csv(f'D:/Jassi/freesound-audio-tagging/dataset2 - Copy.csv')
df1 = df1.drop(['label'],axis=1)
df1

In [ ]:
data = df1.join(df)
data.to_csv(f'D:/Jassi/freesound-audio-tagging/train_new.csv', index=False)

In [ ]:
data = pd.read_csv(r'D:\Jassi\freesound-audio-tagging\train_new.csv')
data['label_id'] = data['label'].factorize()[0]
label_id_data = data[['label', 'label_id']].drop_duplicates().sort_values('label_id')
label_to_id = dict(label_id_data.values)
id_to_label = dict(label_id_data[['label_id', 'label']].values)
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)
data = data.drop(['label'],axis=1)
#Encoding the Labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
#Dividing data into training and Testing set
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)


In [ ]:
from keras.optimizers import SGD
epochs = 200
batch_size = 64
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.99
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

In [ ]:
model = Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],),kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001),bias_regularizer=regularizers.l2(0.001)))
model.add(layers.BatchNormalization())
model.add(Dropout(rate=0.5))

model.add(layers.Dense(64, activation='sigmoid', input_shape=(X_train.shape[1],),kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001),bias_regularizer=regularizers.l2(0.001)))
model.add(layers.BatchNormalization())
model.add(Dropout(rate=0.5))

model.add(layers.Dense(41, activation='softmax',kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001),bias_regularizer=regularizers.l2(0.001)))
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
classifier = model.fit(X_train,
                    y_train,
                    epochs=epochs,
                    validation_data=(X_val,y_val),
                    batch_size=batch_size)

In [ ]:
### print(classifier.history.keys())
#  "Accuracy"
plt.plot(classifier.history['accuracy'])
plt.plot(classifier.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(classifier.history['loss'])
plt.plot(classifier.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("D:\Jassi\freesound-audio-tagging\Audio_600_json.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights('D:\Jassi\freesound-audio-tagging\Audio_600_weights.h5')

model.save('D:\Jassi\freesound-audio-tagging\Audio_600_model.h5')

In [ ]:
#tf.keras.utils.plot_model(
#    model,
#    to_file="model.png",
#    show_shapes=False,
#    show_layer_names=True,
#    rankdir="TB",
#    expand_nested=False,
#    dpi=96,
#)

In [ ]:
prediction = model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
from matplotlib import pyplot
fig, ax = pyplot.subplots(figsize=(15,15))
conf_mat = confusion_matrix(y_test, prediction.argmax(axis=1))
sns.heatmap(conf_mat, annot=True, fmt='d',ax=ax,
            xticklabels=label_id_data.label.values, yticklabels=label_id_data.label.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, prediction.argmax(axis=1))) 

In [ ]:
#practical use
filename = f'D:/Jassi/freesound-audio-tagging/audio_test/00326aa9.wav'
y, sr = librosa.load(filename, mono=True, duration=30)

In [ ]:
rmse = librosa.feature.rms(y=y)
chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
zcr = librosa.feature.zero_crossing_rate(y)
mfcc = librosa.feature.mfcc(y=y, sr=sr)
import wave
nframes = librosa.util.frame(y)


to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)} {np.mean(nframes)}'
sample = np.array([np.mean(chroma_stft), np.mean(rmse), np.mean(spec_cent), 
                     np.mean(spec_bw), np.mean(rolloff), np.mean(zcr), np.mean(nframes)])


for e in mfcc:
    np.mean(e)
sample = np.concatenate((sample, e))

In [ ]:
sample = sample[:27]
sample = sample.reshape(1,27)
sample.shape #(1, 27)

In [ ]:
pred = model.predict(sample)

In [ ]:
import ntpath
for i, predicted in zip(sample, pred.argmax(axis=1)):
    file_name = ntpath.basename(filename)
    print(file_name)
    print("  - Predicted as: '{}'".format(id_to_label[predicted]))
    

In [ ]:
import IPython.display as ipd  # To play sound in the notebook
ipd.Audio(filename)

PART-2

In [ ]:
import pandas as pd
import numpy as np
from numpy import argmax
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import librosa.display
import IPython.display
import random
import warnings
import os
from PIL import Image
import pathlib
from pathlib import Path
import csv
# sklearn Preprocessing
from sklearn.model_selection import train_test_split
#Keras
import keras
import warnings
warnings.filterwarnings('ignore')
from keras import layers
from keras.layers import Activation, Dense, Dropout,LSTM,TimeDistributed,Bidirectional, Conv1D, Conv2D,Lambda, GRU, BatchNormalization, Flatten,MaxPooling1D, MaxPooling2D, GlobalMaxPooling2D, Input, Add
from keras.models import Sequential
from keras.optimizers import SGD
from keras.activations import relu, softmax
from keras import losses, models, optimizers

In [ ]:
import split_folders
#To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
split_folders.ratio(f'D:/Jassi/freesound-audio-tagging/freesound_img_data', output=f"D:/Jassi/freesound-audio-tagging/data", seed=1337, ratio=(.8, .2)) # default values

In [ ]:
paths_all_labels = []
for files in os.listdir(f'D:/Jassi/freesound-audio-tagging/audio_train_labeled'):
    for i in os.listdir(f'D:/Jassi/freesound-audio-tagging/audio_train_labeled/{files}'):
        data_dir = f'D:/Jassi/freesound-audio-tagging/audio_train_labeled/{files}/'
        paths_all_labels += [[data_dir+i] + [files]]
#give output (pth of audio file, label)

In [ ]:
#def load_and_preprocess_lstm(dataset, SAMPLE_SIZE = 30):
#    IMG_SIZE = (216,128) 
#    progress=0

#    data = []
#    labels = []
#    for (path, label) in dataset:
#        audio, sr = librosa.load(path)
#        dur = librosa.get_duration(audio, sr = sr)
#        sampleNum = int(dur / SAMPLE_SIZE)
#        offset = (dur % SAMPLE_SIZE) / 2
#        for i in range(sampleNum):
#            audio, sr = librosa.load(path, offset= offset+i, duration=SAMPLE_SIZE)
##            sample = librosa.feature.melspectrogram(audio, sr=sr)
#           # print(sample.shape)
#            sample = cv2.resize(sample, dsize=IMG_SIZE)
#            data += [(sample, label)]
#            labels += [label]
#        progress +=1
#        print('\r Progress: '+str(round(100*progress/len(dataset))) + '%', end='')
#   return data, labels

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255, # rescale all pixel values from 0-255, so aftre this step all our pixel values are in range (0,1)
        shear_range=0.2, #to apply some random tranfromations
        zoom_range=0.2, #to apply zoom
        horizontal_flip=True) # image will be flipper horiz
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(
        'D:/Jassi/freesound-audio-tagging/data/train',
        target_size=(64, 64),
        batch_size=64,
        class_mode='categorical',
        shuffle = False)
test_set = test_datagen.flow_from_directory(
        'D:/Jassi/freesound-audio-tagging/data/val',
        target_size=(64, 64),
        batch_size=64,
        class_mode='categorical',
        shuffle = False )

In [ ]:


#pre_trained_model = keras.applications.VGG19(include_top=False, weights='imagenet', input_tensor=None,
#                                              input_shape=(64, 64, 3))

#for layer in pre_trained_model.layers:
#    layer.trainable=False
#
#last_layer = pre_trained_model.get_layer('block5_pool')
#last_output = last_layer.output
#inp = Input(shape=(64,64,3))
#x = Conv2D(64, 9, activation=relu, padding="same")(last_output)
#x = Conv2D(64, 9, activation=relu, padding="same")(x)
#x = MaxPooling2D(64,padding='SAME')(x)
#x = Conv2D(128, 9, activation=relu, padding="same")(x)
#x = Conv2D(128, 9, activation=relu, padding="same")(x)
#x = MaxPooling2D(128,padding='SAME')(x)
#x = Conv2D(256, 9, activation=relu, padding="same")(x)
#x = Conv2D(256, 9, activation=relu, padding="same")(x)
#x = Conv2D(256, 9, activation=relu, padding="same")(x)
#x = MaxPooling2D(256,padding='SAME')(x)
#x = Conv2D(512, 9, activation=relu, padding="same")(x)
#x = Conv2D(512, 9, activation=relu, padding="same")(x)
#x = MaxPooling2D(512,padding='SAME')(x)
#x = Conv2D(16, 9, activation=relu, padding="same")(x)
#x = Conv2D(16, 9, activation=relu, padding="same")(x)
#x = MaxPooling2D(16,padding='SAME')(x)
#x = Dropout(rate=0.1)(x)
#x = Conv2D(32, 3, activation=relu, padding="same")(x)
#x = Conv2D(32, 3, activation=relu, padding="same")(x)
##x = GlobalMaxPooling2D()(x)
#x = Dropout(rate=0.1)(x)
#x = Flatten()(x)
#x = Dense(64, activation=relu)(x)
#x = Dense(1028, activation=relu)(x)
#out = Dense(41, activation=softmax)(x)
#model = models.Model(inputs=pre_trained_model.input, outputs=out)
#model.summary()

In [ ]:
from keras import regularizers
pre_trained_model = keras.applications.VGG19(include_top=False, weights='imagenet', input_tensor=None,
                                              input_shape=(64, 64, 3))

for layer in pre_trained_model.layers:
    layer.trainable=False

last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
inp = Input(shape=(64,64,3))
x = Sequential()
x = TimeDistributed(Conv1D(128, 9, activation=relu, padding="same"))(inp)
x = TimeDistributed(MaxPooling1D(128,padding='SAME'))(x)
x = TimeDistributed(keras.layers.BatchNormalization())(x)
x = TimeDistributed(Dropout(rate=0.5))(x)
x = TimeDistributed(Conv1D(64, 9, activation=relu, padding="same"))(x)

x = TimeDistributed(MaxPooling1D(64,padding='SAME'))(x)
x = TimeDistributed(keras.layers.BatchNormalization())(x)
x = TimeDistributed(Dropout(rate=0.5))(x)
x = TimeDistributed(Conv1D(32, 9, activation=relu, padding="same"))(x)

x = TimeDistributed(MaxPooling1D(32,padding='SAME'))(x)
x = TimeDistributed(keras.layers.BatchNormalization())(x)
x = TimeDistributed(Dropout(rate=0.5))(x)

x = TimeDistributed(Flatten())(x)

x = Bidirectional(LSTM(units = 512, dropout=0.5, recurrent_dropout=0.3, return_sequences = True))(x)
x = Bidirectional(LSTM(units = 64, dropout=0.5, recurrent_dropout=0.3, return_sequences = False))(x)
out = Dense(41, activation=softmax)(x)
model = models.Model(inputs=inp, outputs=out)
model.summary()

In [ ]:
epochs = 200
batch_size = 64
learning_rate = 1e-6
decay_rate = learning_rate / epochs
momentum = 0.99
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, TensorBoard
from time import time

# checkpoint
filepath="Audio.best.hdf5"
best_model_checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [best_model_checkpoint]

In [ ]:
model.fit_generator(
        training_set,
        steps_per_epoch=25,
        epochs=200,
        validation_data=test_set,
        validation_steps=200,
        callbacks=callbacks_list)

In [ ]:
print('Done without batch normalization')

In [ ]:
#Model Evaluation
model.evaluate_generator(generator=test_set, steps=50)
#OUTPUT
#[1.704445120342617, 0.33798882681564246]

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("D:\Jassi\freesound-audio-tagging\Audio_json.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights('D:\Jassi\freesound-audio-tagging\Audio_cnn_weights.h5')

model.save('D:\Jassi\freesound-audio-tagging\Audio_cnn_model.h5')

In [ ]:
print('Done')

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_acuracy'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_acuracy'+string])
    plt.show()

    
plot_graphs(model.history, "accuracy")
plot_graphs(model.history, "loss")

In [ ]:
test_set.reset()
pred = model.predict_generator(test_set, steps=50, verbose=1)

In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)

labels = (training_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
predictions = predictions[:210]
filenames=test_set.filenames

In [ ]:
print(len(filenames), len(predictions))
# (200, 200)

In [ ]:
results=pd.DataFrame({"Filename":filenames,"Predictions":predictions})
results.to_csv("prediction_results.csv",index=False)

In [ ]:
#practical use
y, sr = librosa.load(r'D:\Jassi\genres\reggae\reggae.00007.wav', mono=True, duration=30)
plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
plt.axis('off');
plt.savefig(f'D:\Jassi\img.png')
plt.clf()

In [ ]:
img = tf.keras.preprocessing.image.load_img(
    r'D:\Jassi\img.png', grayscale=False, color_mode='rgb', target_size=((64,64)),
    interpolation='nearest')

In [ ]:
img = tf.keras.preprocessing.image.img_to_array(img)
img = img.reshape(1,64,64,3)
img.shape

In [ ]:
pred = model.predict(img)

In [ ]:
import ntpath
file_name = ntpath.basename("D:\Jassi\genres\reggae\reggae.00007.wav")

In [ ]:
for i, predicted in zip(file_name, pred.argmax(axis=1)):
    file_name = ntpath.basename("D:\Jassi\genres\reggae\reggae.00007.wav")
    print(file_name)
    print("  - Predicted as: '{}'".format(id_to_label[predicted]))